In [ ]:
import pandas as pd

df = pd.read_json("/Users/jenishjain/Desktop/workspace/tryouts/output.json")

In [ ]:
df['date'] = pd.to_datetime(df['time'], format='%d-%m-%Y')

# Extract month and year and create new columns
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# df['month'] = df['date'].dt.month
df['month'] = df['date'].dt.strftime('%B')
df['month'] = pd.Categorical(df['month'], categories=month_order, ordered=True)
df['year'] = df['date'].dt.year

def assign_tag(title):
    if title is None:
        return 'unknown'
    tags = {
        'food': ['restaurant', 'cafe', 'dominos', 'swiggy', 'food', 'zomato' , 'staples enterprise', 'master harsh mitruka', 'icecream'],
        'travel': ['flight', 'hotel', 'uber', 'ola', 'rapido', 'travel', 'irctc', 'makemytrip', 'goibibo', 'ontrot', 'bus'],
        'clothes': ['myntra', 'zara', 'levis', 'gap', 'espanshe'],
        'investment': ['mutual fund', 'stock', 'investment', 'sip', 'groww', 'zerodha'],
        'grocery': ['bigbasket', 'grofers', 'grocery', 'zepto', 'hyper mart', 'bakery', 'supermarket', 'dairy'],
        'entertainment': ['bookmyshow', 'netflix', 'prime', 'hotstar', 'spotify', 'gaana', 'music', 'entertainment'],
        'shopping': ['amazon', 'flipkart', 'snapdeal', 'shopping', 'shop', 'store'],
        'selfcare': ['salon', 'spa', 'selfcare', 'gym', 'fitness', 'health', 'medical'],
        'bills' : ['electricity', 'water', 'bills', 'bill', 'dth', 'internet', 'phone', 'recharge', 'gas'],
    }

    for tag, keywords in tags.items():
        if any(keyword in title.lower() for keyword in keywords):
            return tag

    return 'miscellaneous'
df['category'] = df['title'].apply(assign_tag)

In [ ]:
df.head()

In [ ]:
completed_transactions = df[df['status'] == 'Completed']
completed_transactions['amount'] = completed_transactions['amount'].fillna(0)
completed_transactions.head()

completed_transactions = completed_transactions.sort_values(['year', 'month'], ascending=False)

In [ ]:
total_lifetime_spend = completed_transactions['amount'].sum()

total_lifetime_spend

In [ ]:
total_expense_group_by_month_and_year = completed_transactions.groupby(['year', 'month'])['amount'].sum().reset_index()

total_expense_group_by_month_and_year = total_expense_group_by_month_and_year.sort_values(['year', 'month'], ascending=False)

total_expense_group_by_month_and_year.plot(kind='bar', x='month', y='amount', figsize=(15, 6))

In [ ]:
filter_data_for_year = completed_transactions[completed_transactions['year'] == 2024]

In [ ]:
grouped_expenses = filter_data_for_year.groupby(['year', 'month', 'category'])['amount'].sum()

grouped_expenses = grouped_expenses.reset_index()

# grouped_expenses = grouped_expenses.sort_values(['year', 'month'], ascending=False)

grouped_expenses

# grouped_expenses.plot(kind='bar', x='category', y='amount', figsize=(15, 6))

In [ ]:

# Pivot the DataFrame
pivot_df = grouped_expenses.pivot_table(index=['year', 'month'], columns='category', values='amount', fill_value=0)

# Plot the stacked bar chart
pivot_df.plot(kind='bar', stacked=True, figsize=(15, 6))

In [ ]:
data_without_unknown_and_miscellaneous = grouped_expenses[(grouped_expenses['category'] != 'unknown') & (grouped_expenses['category'] != 'miscellaneous')]
pivot_df = data_without_unknown_and_miscellaneous.pivot_table(index=['year', 'month'], columns='category', values='amount', fill_value=0)
pivot_df.plot(kind='bar', stacked=True, figsize=(15, 6))
